In [1]:
import sqlite3

conn = sqlite3.connect('spider-new-url.sqlite')
cur = conn.cursor()

# Find the ids that send out page rank - we only are interested
# in pages in the SCC that have in and out links
cur.execute('''SELECT DISTINCT from_id FROM Links''')
from_ids = list()
for row in cur: 
    from_ids.append(row[0])

# Find the ids that receive page rank 
to_ids = list()
links = list()
cur.execute('''SELECT DISTINCT from_id, to_id FROM Links''')
for row in cur:
    from_id = row[0]
    to_id = row[1]
    if from_id == to_id : continue
    if from_id not in from_ids : continue
    if to_id not in from_ids : continue
    links.append(row)
    if to_id not in to_ids : to_ids.append(to_id)


# Get latest page ranks for strongly connected component
prev_ranks = dict()
for node in from_ids:
    cur.execute('''SELECT new_rank FROM Pages WHERE id = ?''', (node, ))
    row = cur.fetchone()
    prev_ranks[node] = row[0]

sval = input('How many iterations:')
many = 1
if ( len(sval) > 0 ) : many = int(sval)


# Sanity check
if len(prev_ranks) < 1 : 
    print("Nothing to page rank.  Check data.")
    quit()



# Lets do Page Rank in memory so it is really fast
for i in range(many):
    # print prev_ranks.items()[:5]
    next_ranks = dict();
    total = 0.0
    for (node, old_rank) in list(prev_ranks.items()):
        total = total + old_rank
        next_ranks[node] = 0.0
    # print total

    # Find the number of outbound links and sent the page rank down each
    for (node, old_rank) in list(prev_ranks.items()):
        # print node, old_rank
        give_ids = list()
        for (from_id, to_id) in links:
            if from_id != node : continue
           #  print '   ',from_id,to_id

            if to_id not in to_ids: continue
            give_ids.append(to_id)
        if ( len(give_ids) < 1 ) : continue
        amount = old_rank / len(give_ids)
        # print node, old_rank,amount, give_ids
    
        for id in give_ids:
            next_ranks[id] = next_ranks[id] + amount
    
    newtot = 0
    for (node, next_rank) in list(next_ranks.items()):
        newtot = newtot + next_rank
    evap = (total - newtot) / len(next_ranks)

    # print newtot, evap
    for node in next_ranks:
        next_ranks[node] = next_ranks[node] + evap

    newtot = 0
    for (node, next_rank) in list(next_ranks.items()):
        newtot = newtot + next_rank

    # Compute the per-page average change from old rank to new rank
    # As indication of convergence of the algorithm
    totdiff = 0
    for (node, old_rank) in list(prev_ranks.items()):
        new_rank = next_ranks[node]
        diff = abs(old_rank-new_rank)
        totdiff = totdiff + diff

    avediff = totdiff / len(prev_ranks)
    print(i+1, avediff)

    # rotate
    prev_ranks = next_ranks


# Put the final ranks back into the database
print(list(next_ranks.items())[:5])

cur.execute('''UPDATE Pages SET old_rank=new_rank''')

for (id, new_rank) in list(next_ranks.items()) :
    cur.execute('''UPDATE Pages SET new_rank=? WHERE id=?''', (new_rank, id))


conn.commit()

cur.close()

How many iterations:100
1 1.0775000000000006
2 1.0328796296296294
3 0.628047916666666
4 0.43690815972222347
5 0.2993081801911835
6 0.18055478078917686
7 0.10429374098841564
8 0.06326740087009737
9 0.03662279544651825
10 0.021808942140723295
11 0.01267245003153858
12 0.007451145246741835
13 0.004360735005875151
14 0.002549112715151072
15 0.001582257771341037
16 0.0009325109488945464
17 0.0005793682339942602
18 0.00034185824038800475
19 0.00021324962632087826
20 0.0001252596566715602
21 7.800525209083442e-05
22 4.566365972804899e-05
23 2.8386958275726178e-05
24 1.656097204219005e-05
25 1.0299964197900134e-05
26 5.9957099104280165e-06
27 3.7252131648787952e-06
28 2.1644721567922638e-06
29 1.3446105419732047e-06
30 7.791536343781922e-07
31 4.837128058965099e-07
32 2.7970560438840346e-07
33 1.7356141090893936e-07
34 1.0011778857071454e-07
35 6.209219930852008e-08
36 3.5737992588855825e-08
37 2.2152620613161146e-08
38 1.272086133414299e-08
39 7.881468246957735e-09
40 4.515501932753768e-09
41